In [2]:
pip install pinecone

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [4]:
pip install -U langchain-community


Note: you may need to restart the kernel to use updated packages.


In [5]:
PINECONE_API_KEY = "pcsk_7Myivb_KdeTtvUZmMKfB2fpVaixRPYyhzjfVJnxprdGtxB7pSpKnnXXCja3bNYHUqRnegj"
PINECONE_API_ENV = "us-east-1"

In [6]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [7]:
extracted_data = load_pdf("/Users/nandinigadkol/Desktop/End-to-end-Medical-Chatbot-using-Llama2/data")

In [7]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [8]:
# extracted_data

In [9]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [11]:
# text_chunks

In [11]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

/var/folders/7k/tdvsqyyx5tj22_6kjnz3c4c40000gn/T/ipykernel_75025/1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [13]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [14]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [15]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [16]:
# query_result

In [18]:
import os

# Set the API key directly in the Python script
os.environ["PINECONE_API_KEY"] = "pcsk_7Myivb_KdeTtvUZmMKfB2fpVaixRPYyhzjfVJnxprdGtxB7pSpKnnXXCja3bNYHUqRnegj"

# Now, print the API key to verify
print(os.getenv("PINECONE_API_KEY"))

pcsk_7Myivb_KdeTtvUZmMKfB2fpVaixRPYyhzjfVJnxprdGtxB7pSpKnnXXCja3bNYHUqRnegj


In [19]:
from pinecone import Pinecone, ServerlessSpec
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize Pinecone client with the correct API key and environment
pinecone_client = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

# Initialize the index name and embeddings
index_name = "aihealthcarechatbot"

# Check if the index already exists
if index_name not in pinecone_client.list_indexes():
    pinecone_client.create_index(
        name=index_name,
        dimension=768,  # Match the embedding size
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
else:
    print(f"Index '{index_name}' already exists. Skipping creation.")


# Initialize embeddings
embeddings = HuggingFaceEmbeddings()

# Create the Pinecone vector store from the text chunks and embeddings
docsearch = LangchainPinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


/var/folders/7k/tdvsqyyx5tj22_6kjnz3c4c40000gn/T/ipykernel_75025/3397664757.py:27: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [20]:
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings()

# Test the embedding dimensionality
sample_embedding = embedding_model.embed_query("Test query")
print(f"Embedding size: {len(sample_embedding)}")  # Should print the dimension


/var/folders/7k/tdvsqyyx5tj22_6kjnz3c4c40000gn/T/ipykernel_75025/1065050653.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding_model = HuggingFaceEmbeddings()


Embedding size: 768


In [21]:
# List all existing indexes
print("Existing indexes:", pinecone_client.list_indexes())



Existing indexes: {'indexes': [{'deletion_protection': 'disabled',
              'dimension': 2,
              'host': 'medical-chatbot-19b1a0j.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'medical-chatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'deletion_protection': 'disabled',
              'dimension': 768,
              'host': 'aihealthcarechatbot-19b1a0j.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'aihealthcarechatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


In [22]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [23]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [24]:
llm=CTransformers(model="/Users/nandinigadkol/Desktop/End-to-end-Medical-Chatbot-using-Llama2/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [25]:
pip install ctransformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [26]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [27]:
while True:
    # Accept user query
    user_query = input("Enter your query (or type 'exit' to quit): ")
    
    # Exit condition
    if user_query.lower() == "exit":
        print("Exiting... Goodbye!")
        break
    
    # Process the query using the RetrievalQA pipeline
    try:
        response = qa(user_query)
        # Print the answer
        print("\nAnswer:", response['result'])
        
        # Optionally, print source documents
        if 'source_documents' in response:
            print("\nSources:")
            for doc in response['source_documents']:
                print(doc.metadata.get('source', 'Unknown'), ":", doc.page_content[:200], "...")  # Truncated content
    except Exception as e:
        print(f"An error occurred: {e}")


/var/folders/7k/tdvsqyyx5tj22_6kjnz3c4c40000gn/T/ipykernel_75025/2586122779.py:12: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa(user_query)



Answer: There are several acupressure points that can be used to relieve pressure on the shoulders. Here are a few options:

1. The "Third Eye" point: Located between the eyebrows, this point is said to help alleviate tension and stress in the body.
2. The "Shoulder Well" point: Located on the shoulder blade, this point is said to help relax the muscles in the shoulders and neck.
3. The "Liver 3" point: Located on the upper back, this point is said to help reduce tension in the liver and gallbladder, which can contribute to pressure on the shoulders.

I hope this helps! Let me know if you have any other questions.

Sources:
Unknown : Acupressure
Therapist working acupressure points on a woman’s shoulder.(Photo Researchers, Inc. Reproduced by permission.)
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 36 ...
Unknown : • a variable-pressure mattress whose sections can be
individually inflated or deflated to redistribute pressure
Pillows or foam wedges can prevent a bedridden
patient’s a

In [29]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sentence_transformers import SentenceTransformer, util
from collections import Counter
from itertools import chain
import numpy as np
import math

# Load pre-trained model for cosine similarity
model_cosine = SentenceTransformer('all-MiniLM-L6-v2')

# Function to calculate cosine similarity
def calculate_cosine_similarity(reference, response):
    """
    Calculates cosine similarity between reference and response texts.
    :param reference: Reference text (string).
    :param response: Model-generated response (string).
    :return: Cosine similarity score as a float.
    """
    ref_embedding = model_cosine.encode(reference, convert_to_tensor=True)
    resp_embedding = model_cosine.encode(response, convert_to_tensor=True)
    similarity_score = util.cos_sim(ref_embedding, resp_embedding).item()
    return similarity_score

# BLEU Score Calculation
def calculate_bleu(reference, candidate, max_n=2):
    """
    Calculates BLEU score from scratch.
    :param reference: The reference text (string).
    :param candidate: The candidate text generated by the model (string).
    :param max_n: Maximum n-gram order to consider (default is bigram, 2).
    :return: BLEU score as a float.
    """
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    precisions = []
    for n in range(1, max_n + 1):
        ref_ngrams = Counter(tuple(reference_tokens[i:i + n]) for i in range(len(reference_tokens) - n + 1))
        cand_ngrams = Counter(tuple(candidate_tokens[i:i + n]) for i in range(len(candidate_tokens) - n + 1))
        match_count = sum(min(count, cand_ngrams[ngram]) for ngram, count in ref_ngrams.items())
        total_count = sum(cand_ngrams.values())
        precision = match_count / total_count if total_count > 0 else 0
        precisions.append(precision)

    if all(p > 0 for p in precisions):
        geometric_mean = math.exp(sum(math.log(p) for p in precisions) / max_n)
    else:
        geometric_mean = 0

    ref_length = len(reference_tokens)
    cand_length = len(candidate_tokens)
    brevity_penalty = math.exp(1 - ref_length / cand_length) if cand_length > 0 and cand_length < ref_length else 1

    bleu_score = brevity_penalty * geometric_mean
    return bleu_score

# ROUGE-N Calculation
def calculate_rouge_n(reference, response, n=1):
    """
    Calculate ROUGE-N (precision, recall, F1-score).
    :param reference: Reference text (string).
    :param response: Model-generated text (string).
    :param n: N-gram size (default is 1, unigram).
    :return: Precision, Recall, and F1-score.
    """
    reference_tokens = reference.split()
    response_tokens = response.split()
    ref_ngrams = Counter(tuple(reference_tokens[i:i + n]) for i in range(len(reference_tokens) - n + 1))
    resp_ngrams = Counter(tuple(response_tokens[i:i + n]) for i in range(len(response_tokens) - n + 1))
    overlap = sum(min(ref_ngrams[ngram], resp_ngrams[ngram]) for ngram in ref_ngrams)
    ref_total = sum(ref_ngrams.values())
    resp_total = sum(resp_ngrams.values())
    precision = overlap / resp_total if resp_total > 0 else 0
    recall = overlap / ref_total if ref_total > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return precision, recall, f1

# ROUGE-L Calculation
def calculate_rouge_l(reference, response):
    """
    Calculate ROUGE-L (Longest Common Subsequence).
    :param reference: Reference text (string).
    :param response: Model-generated text (string).
    :return: Precision, Recall, and F1-score.
    """
    reference_tokens = reference.split()
    response_tokens = response.split()

    def lcs_length(x, y):
        dp = [[0] * (len(y) + 1) for _ in range(len(x) + 1)]
        for i in range(1, len(x) + 1):
            for j in range(1, len(y) + 1):
                if x[i - 1] == y[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1] + 1
                else:
                    dp[i][j] = max(dp[i - 1][j], dp[i][j - 1])
        return dp[-1][-1]

    lcs_len = lcs_length(reference_tokens, response_tokens)
    precision = lcs_len / len(response_tokens) if len(response_tokens) > 0 else 0
    recall = lcs_len / len(reference_tokens) if len(reference_tokens) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    return precision, recall, f1

# Closest Reference Finder
def get_closest_reference(user_query, reference_answers):
    """
    Find the closest reference answer for the given user query.
    :param user_query: The query input by the user.
    :param reference_answers: A dictionary of reference answers.
    :return: Closest matching reference answer as a string.
    """
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    question_embedding = model.encode([user_query])
    reference_embeddings = model.encode(list(reference_answers.keys()))
    similarities = np.dot(question_embedding, reference_embeddings.T)
    most_similar_idx = np.argmax(similarities)
    return list(reference_answers.values())[most_similar_idx] if similarities[0][most_similar_idx] > 0 else None

# Evaluation Workflow
def evaluate_llama_response(user_query, reference_answers):
    """
    Generate a response from LLaMA and evaluate it against reference answers.
    :param user_query: Query input by the user (string).
    :param reference_answers: Dictionary of reference answers.
    :return: Dictionary of metrics and the generated response.
    """
    result = qa.run(user_query)  # Assuming `qa` is your RetrievalQA chain
    generated_response = result['result']

    closest_reference = get_closest_reference(user_query, reference_answers)
    if closest_reference is None:
        print("No relevant reference found for the query.")
        return None

    print(f"User Query: {user_query}")
    print(f"Generated Response: {generated_response}")
    print(f"Reference Answer: {closest_reference}")

    cosine_score = calculate_cosine_similarity(closest_reference, generated_response)
    bleu_score = calculate_bleu(closest_reference, generated_response)
    rouge_precision, rouge_recall, rouge_f1 = calculate_rouge_n(closest_reference, generated_response, n=1)
    rouge_l_precision, rouge_l_recall, rouge_l_f1 = calculate_rouge_l(closest_reference, generated_response)

    print("\nEvaluation Metrics:")
    print(f"Cosine Similarity: {cosine_score:.4f}")
    print(f"BLEU Score: {bleu_score:.4f}")
    print(f"ROUGE-1 -> Precision: {rouge_precision:.4f}, Recall: {rouge_recall:.4f}, F1: {rouge_f1:.4f}")
    print(f"ROUGE-L -> Precision: {rouge_l_precision:.4f}, Recall: {rouge_l_recall:.4f}, F1: {rouge_l_f1:.4f}")

    return {
        "query": user_query,
        "generated_response": generated_response,
        "reference_answer": closest_reference,
        "cosine_similarity": cosine_score,
        "bleu_score": bleu_score,
        "rouge_1": {"precision": rouge_precision, "recall": rouge_recall, "f1": rouge_f1},
        "rouge_l": {"precision": rouge_l_precision, "recall": rouge_l_recall, "f1": rouge_l_f1}
    }


In [31]:
from references import reference_answers  # Replace 'reference_answers' with your file name

# Function to evaluate LLaMA responses
def evaluate_llama_response(user_query):
    """
    Generate a response from LLaMA and evaluate it against reference answers.
    :param user_query: Query input by the user (string).
    :return: Dictionary of metrics and the generated response.
    """
    # Generate a response from the LLaMA model
    result = qa({"query": user_query})  # Use the `__call__` method to handle multiple output keys
    generated_response = result["result"]  # Access the "result" key for the generated response

    # Find the closest reference answer
    closest_reference = get_closest_reference(user_query, reference_answers)
    if closest_reference is None:
        print("No relevant reference found for the query.")
        return None

    # Calculate evaluation metrics
    cosine_score = calculate_cosine_similarity(closest_reference, generated_response)
    bleu_score = calculate_bleu(closest_reference, generated_response)
    rouge_precision, rouge_recall, rouge_f1 = calculate_rouge_n(closest_reference, generated_response, n=1)
    rouge_l_precision, rouge_l_recall, rouge_l_f1 = calculate_rouge_l(closest_reference, generated_response)

    return {
        "query": user_query,
        "cosine_similarity": cosine_score,
        "bleu_score": bleu_score,
        "rouge_1": {"precision": rouge_precision, "recall": rouge_recall, "f1": rouge_f1},
        "rouge_l": {"precision": rouge_l_precision, "recall": rouge_l_recall, "f1": rouge_l_f1}
    }

# Function to evaluate multiple queries and calculate averages
def evaluate_multiple_queries_with_averages(user_queries):
    """
    Evaluate multiple queries using the LLaMA model and calculate average metrics.
    :param user_queries: List of user queries to evaluate.
    :return: List of evaluation results and average metrics.
    """
    results = []
    total_metrics = {
        "cosine_similarity": 0,
        "bleu_score": 0,
        "rouge_1_f1": 0,
        "rouge_l_f1": 0,
        "rouge_1_recall": 0,
        "rouge_l_recall": 0
    }
    num_queries = len(user_queries)

    for query in user_queries:
        print(f"\nEvaluating query: {query}")
        result = evaluate_llama_response(query)
        if result:
            results.append(result)
            # Accumulate metrics for averaging
            total_metrics["cosine_similarity"] += result["cosine_similarity"]
            total_metrics["bleu_score"] += result["bleu_score"]
            total_metrics["rouge_1_f1"] += result["rouge_1"]["f1"]
            total_metrics["rouge_l_f1"] += result["rouge_l"]["f1"]
            total_metrics["rouge_1_recall"] += result["rouge_1"]["recall"]
            total_metrics["rouge_l_recall"] += result["rouge_l"]["recall"]

    # Calculate average metrics
    average_metrics = {key: value / num_queries for key, value in total_metrics.items()}

    return results, average_metrics

# Example usage
if __name__ == "__main__":
    user_queries = [
        "What is acupressure?",
        "What is angiography?",
        "What is anosmia?",
        "How is anosmia treated?",
        "What causes alopecia?",
        "What is male pattern baldness?",
        "What are the symptoms of anosmia?",
        "What is the purpose of angiography?",
        "How is alopecia diagnosed?",
        "Can alopecia be cured?"
    ]

    evaluation_results, average_metrics = evaluate_multiple_queries_with_averages(user_queries)

    # Print a summary of the evaluation results
    for result in evaluation_results:
        print("\n" + "=" * 40)
        print(f"Query: {result['query']}")
        print("Metrics:")
        print(f"  Cosine Similarity: {result['cosine_similarity']:.4f}")
        print(f"  BLEU Score: {result['bleu_score']:.4f}")
        print(f"  ROUGE-1 F1: {result['rouge_1']['f1']:.4f}")
        print(f"  ROUGE-1 Recall: {result['rouge_1']['recall']:.4f}")
        print(f"  ROUGE-L F1: {result['rouge_l']['f1']:.4f}")
        print(f"  ROUGE-L Recall: {result['rouge_l']['recall']:.4f}")

    # Print average metrics
    print("\n" + "=" * 40)
    print("Average Metrics Across All Queries:")
    print(f"  Average Cosine Similarity: {average_metrics['cosine_similarity']:.4f}")
    print(f"  Average BLEU Score: {average_metrics['bleu_score']:.4f}")
    print(f"  Average ROUGE-1 F1: {average_metrics['rouge_1_f1']:.4f}")
    print(f"  Average ROUGE-1 Recall: {average_metrics['rouge_1_recall']:.4f}")
    print(f"  Average ROUGE-L F1: {average_metrics['rouge_l_f1']:.4f}")
    print(f"  Average ROUGE-L Recall: {average_metrics['rouge_l_recall']:.4f}")



Evaluating query: What is acupressure?

Evaluating query: What is angiography?

Evaluating query: What is anosmia?

Evaluating query: How is anosmia treated?

Evaluating query: What causes alopecia?

Evaluating query: What is male pattern baldness?

Evaluating query: What are the symptoms of anosmia?

Evaluating query: What is the purpose of angiography?

Evaluating query: How is alopecia diagnosed?

Evaluating query: Can alopecia be cured?

Query: What is acupressure?
Metrics:
  Cosine Similarity: 0.9644
  BLEU Score: 0.1969
  ROUGE-1 F1: 0.5517
  ROUGE-1 Recall: 0.3810
  ROUGE-L F1: 0.5517
  ROUGE-L Recall: 0.3810

Query: What is angiography?
Metrics:
  Cosine Similarity: 0.9537
  BLEU Score: 0.2795
  ROUGE-1 F1: 0.4158
  ROUGE-1 Recall: 0.5676
  ROUGE-L F1: 0.3960
  ROUGE-L Recall: 0.5405

Query: What is anosmia?
Metrics:
  Cosine Similarity: 0.8383
  BLEU Score: 0.1633
  ROUGE-1 F1: 0.2821
  ROUGE-1 Recall: 0.2292
  ROUGE-L F1: 0.2308
  ROUGE-L Recall: 0.1875

Query: How is anosmi

In [33]:
from references import reference_answers  # Replace 'reference_answers' with your file name

# Function to evaluate LLaMA responses
def evaluate_llama_response(user_query):
    """
    Generate a response from LLaMA and evaluate it against reference answers.
    :param user_query: Query input by the user (string).
    :return: Dictionary of metrics and the generated response.
    """
    # Generate a response from the LLaMA model
    result = qa({"query": user_query})  # Use the `__call__` method to handle multiple output keys
    generated_response = result["result"]  # Access the "result" key for the generated response

    # Find the closest reference answer
    closest_reference = get_closest_reference(user_query, reference_answers)
    if closest_reference is None:
        print("No relevant reference found for the query.")
        return None

    # Calculate evaluation metrics
    cosine_score = calculate_cosine_similarity(closest_reference, generated_response)
    bleu_score = calculate_bleu(closest_reference, generated_response)
    rouge_precision, rouge_recall, rouge_f1 = calculate_rouge_n(closest_reference, generated_response, n=1)
    rouge_l_precision, rouge_l_recall, rouge_l_f1 = calculate_rouge_l(closest_reference, generated_response)

    return {
        "query": user_query,
        "cosine_similarity": cosine_score,
        "bleu_score": bleu_score,
        "rouge_1": {"precision": rouge_precision, "recall": rouge_recall, "f1": rouge_f1},
        "rouge_l": {"precision": rouge_l_precision, "recall": rouge_l_recall, "f1": rouge_l_f1}
    }

# Function to evaluate multiple queries and calculate averages
def evaluate_multiple_queries_with_averages(user_queries):
    """
    Evaluate multiple queries using the LLaMA model and calculate average metrics.
    :param user_queries: List of user queries to evaluate.
    :return: List of evaluation results and average metrics.
    """
    results = []
    total_metrics = {
        "cosine_similarity": 0,
        "bleu_score": 0,
        "rouge_1_f1": 0,
        "rouge_l_f1": 0
    }
    num_queries = len(user_queries)

    for query in user_queries:
        print(f"\nEvaluating query: {query}")
        result = evaluate_llama_response(query)
        if result:
            results.append(result)
            # Accumulate metrics for averaging
            total_metrics["cosine_similarity"] += result["cosine_similarity"]
            total_metrics["bleu_score"] += result["bleu_score"]
            total_metrics["rouge_1_f1"] += result["rouge_1"]["f1"]
            total_metrics["rouge_l_f1"] += result["rouge_l"]["f1"]

    # Calculate average metrics
    average_metrics = {key: value / num_queries for key, value in total_metrics.items()}

    return results, average_metrics

# Example usage
if __name__ == "__main__":
    user_queries = [
        "What is acupressure?",
        "What is angiography?",
        "What is anosmia?",
        "How is anosmia treated?",
        "What causes alopecia?",
        "What is male pattern baldness?",
        "What are the symptoms of anosmia?",
        "What is the purpose of angiography?",
        "How is alopecia diagnosed?",
        "Can alopecia be cured?"
    ]

    evaluation_results, average_metrics = evaluate_multiple_queries_with_averages(user_queries)

    # Print a summary of the evaluation results
    for result in evaluation_results:
        print("\n" + "=" * 40)
        print(f"Query: {result['query']}")
        print("Metrics:")
        print(f"  Cosine Similarity: {result['cosine_similarity']:.4f}")
        print(f"  BLEU Score: {result['bleu_score']:.4f}")
        print(f"  ROUGE-1 F1: {result['rouge_1']['f1']:.4f}")
        print(f"  ROUGE-L F1: {result['rouge_l']['f1']:.4f}")

    # Print average metrics
    print("\n" + "=" * 40)
    print("Average Metrics Across All Queries:")
    print(f"  Average Cosine Similarity: {average_metrics['cosine_similarity']:.4f}")
    print(f"  Average BLEU Score: {average_metrics['bleu_score']:.4f}")
    print(f"  Average ROUGE-1 F1: {average_metrics['rouge_1_f1']:.4f}")
    print(f"  Average ROUGE-L F1: {average_metrics['rouge_l_f1']:.4f}")



Evaluating query: What is acupressure?

Evaluating query: What is angiography?

Evaluating query: What is anosmia?

Evaluating query: How is anosmia treated?

Evaluating query: What causes alopecia?

Evaluating query: What is male pattern baldness?

Evaluating query: What are the symptoms of anosmia?

Evaluating query: What is the purpose of angiography?

Evaluating query: How is alopecia diagnosed?

Evaluating query: Can alopecia be cured?

Query: What is acupressure?
Metrics:
  Cosine Similarity: 0.9644
  BLEU Score: 0.1969
  ROUGE-1 F1: 0.5517
  ROUGE-L F1: 0.5517

Query: What is angiography?
Metrics:
  Cosine Similarity: 0.9236
  BLEU Score: 0.0446
  ROUGE-1 F1: 0.3913
  ROUGE-L F1: 0.3913

Query: What is anosmia?
Metrics:
  Cosine Similarity: 0.8603
  BLEU Score: 0.0158
  ROUGE-1 F1: 0.3103
  ROUGE-L F1: 0.2759

Query: How is anosmia treated?
Metrics:
  Cosine Similarity: 0.3321
  BLEU Score: 0.0951
  ROUGE-1 F1: 0.2400
  ROUGE-L F1: 0.1600

Query: What causes alopecia?
Metrics:
